In [1]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql
dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql


In [2]:
all_tags = pl.DataFrame(stash.find_tags(fragment="id name"))

ai_tags = all_tags.filter(pl.col("name").str.ends_with("_AI"))
ai_tag_ids = ai_tags.select("id").to_series().to_list()
ai_tags

id,name
str,str
"""8867""","""1F1M_AI"""
"""8886""","""1F2M_AI"""
"""8876""","""1F_AI"""
"""8880""","""1M_AI"""
"""8883""","""2F1M_AI"""
…,…
"""8637""","""Wet (Genitals)_AI"""
"""8652""","""Wet (Water)_AI"""
"""8765""","""Wet Pussy_AI"""


In [21]:
stash_tags = stash_client.get_tags_by_names(["AI", "AI_Tagged", "Pussy Closeup_AI", "Video Cut"])

🏷️  Tag Lookup Results:
✅ ai                             -> AI (ID: 8552)
✅ ai_tagged                      -> AI_Tagged (ID: 8559)
✅ pussy_closeup_ai               -> Pussy Closeup_AI (ID: 8768)
✅ video_cut                      -> Video Cut (ID: 8836)
Found 4 out of 4 tags


In [70]:
# Get scenes that have both AI_Tagged and Pussy Closeup_AI tags
scenes_with_markers = stash.find_scenes({
    "tags": {
        "value": [stash_tags.ai_tagged["id"]],
        "modifier": "INCLUDES_ALL",
    },
}, { "sort": "id", "per_page": 1000, "page": 2 }, fragment="id title tags { id name parents { id name } } scene_markers { id primary_tag { id name } }")

# Convert to DataFrame and extract marker information
scenes_df = pl.DataFrame(scenes_with_markers)

# Create a DataFrame with all scene markers
markers_df = scenes_df.explode("scene_markers").with_columns([
    pl.col("scene_markers").struct.field("primary_tag").struct.field("id").alias("primary_tag_id"),
    pl.col("scene_markers").struct.field("primary_tag").struct.field("name").alias("primary_tag_name"),
])

# Create two separate dataframes - one for AI tags and one for all tags
ai_markers_df = markers_df.filter(pl.col("primary_tag_name").str.ends_with("_AI"))
all_markers_df = markers_df.filter(pl.col("primary_tag_id") != stash_tags.video_cut["id"])

# Group by scene and calculate unique AI tags
ai_tag_counts = (ai_markers_df
    .group_by(["id", "title", "tags"])
    .agg([
        pl.col("primary_tag_id").n_unique().alias("unique_ai_tags"),
        pl.col("primary_tag_name").alias("ai_tag_names")
    ])
    .with_columns([
        pl.col("ai_tag_names").list.unique().alias("ai_tag_names")
    ])
    .sort("unique_ai_tags")
)

# Group by scene and get all marker tags
all_tag_counts = (all_markers_df
    .group_by(["id", "title"])
    .agg([
        pl.col("primary_tag_id").n_unique().alias("unique_all_tags"),
        pl.col("primary_tag_name").alias("all_tag_names")
    ])
    .with_columns([
        pl.col("all_tag_names").list.unique().alias("all_tag_names")
    ])
)

# Join the results and format both columns
scenes_sort_by_marker_count = (ai_tag_counts
    .join(all_tag_counts, on=["id", "title"])
    .with_columns([
        pl.col("ai_tag_names").map_elements(lambda x: ", ".join(sorted(x)), return_dtype=pl.Utf8).alias("ai_tags_formatted"),
        pl.col("all_tag_names").map_elements(lambda x: ", ".join(sorted(x)), return_dtype=pl.Utf8).alias("all_marker_tags_formatted")
    ])
    .drop(["ai_tag_names", "all_tag_names"])
).sort("unique_ai_tags")
scenes_sort_by_marker_count


id,title,tags,unique_ai_tags,unique_all_tags,ai_tags_formatted,all_marker_tags_formatted
str,str,list[struct[3]],u32,u32,str,str
"""18428""","""Senora and Jorge""","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""8621"",""Ass_AI"",[{""8552"",""AI""}]}, … {""7511"",""Wavy Hair"",[{""7770"",""Category: Hair Style""}]}]",2,2,"""Cum on Pussy_AI, Pussy Very Cl…","""Cum on Pussy_AI, Pussy Very Cl…"
"""14291""","""Morning Romance""","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""5093"",""American"",[{""7752"",""Category: Misc""}]}, … {""7522"",""Wet Look"",[{""7751"",""Category: Themes""}]}]",2,2,"""Pussy Very Closeup_AI, Wet (Wa…","""Pussy Very Closeup_AI, Wet (Wa…"
"""14654""","""Weekend - Episode 4 - Time For…","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""5169"",""Ass Grabbing"",[{""7753"",""Category: Acts""}]}, … {""7511"",""Wavy Hair"",[{""7770"",""Category: Hair Style""}]}]",2,2,"""Cum on Pussy_AI, Pussy Very Cl…","""Cum on Pussy_AI, Pussy Very Cl…"
"""19444""","""Boss and Employee Celebrate wi…","[{""5054"",""4K Available"",[{""7752"",""Category: Misc""}]}, {""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, … {""6146"",""Woman's Heels"",[{""7762"",""Category: Clothing""}]}]",2,2,"""Cum_AI, Pussy Very Closeup_AI""","""Cum_AI, Pussy Very Closeup_AI"""
"""22436""","""Sun Salutation""","[{""5054"",""4K Available"",[{""7752"",""Category: Misc""}]}, {""5059"",""60 FPS"",[{""7749"",""Category: Shot Type""}]}, … {""7549"",""Workout"",[{""7751"",""Category: Themes""}]}]",2,2,"""Pussy Rubbing_AI, Vaginal Pene…","""Pussy Rubbing_AI, Vaginal Pene…"
…,…,…,…,…,…,…
"""13883""","""Flash Photography""","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""5101"",""Anal Creampie"",[{""7761"",""Category: Finishers""}, {""5558"",""Creampie""}, … {""8848"",""Marker Group: 6. Cumshot""}]}, … {""8661"",""Whip_AI"",[{""8552"",""AI""}]}]",32,42,"""Anal Fucking_AI, Ass Licking_A…","""Anal Creampie, Anal Fucking_AI…"
"""17682""","""4-Player Games""","[{""8867"",""1F1M_AI"",[{""8552"",""AI""}, {""8895"",""AI Group Makeup""}]}, {""5060"",""69"",[{""7753"",""Category: Acts""}]}, … {""7536"",""White Woman"",[{""7764"",""Category: Race""}]}]",34,37,"""1F1M_AI, 1F2M_AI, 1M_AI, 2F1M_…","""1F1M_AI, 1F2M_AI, 1M_AI, 2F1M_…"
"""14357""","""Crunch Time""","[{""8641"",""69_AI"",[{""8552"",""AI""}]}, {""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, … {""8765"",""Wet Pussy_AI"",[{""8552"",""AI""}]}]",35,41,"""Anal Fucking_AI, Anal Gape_AI,…","""Anal Fucking_AI, Anal Gape_AI,…"


In [71]:
scenes_to_be_reset = scenes_sort_by_marker_count.filter(
    pl.col("unique_ai_tags") == pl.col("unique_all_tags"),
    pl.col("unique_ai_tags") == 2
)
scenes_to_be_reset

id,title,tags,unique_ai_tags,unique_all_tags,ai_tags_formatted,all_marker_tags_formatted
str,str,list[struct[3]],u32,u32,str,str
"""18428""","""Senora and Jorge""","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""8621"",""Ass_AI"",[{""8552"",""AI""}]}, … {""7511"",""Wavy Hair"",[{""7770"",""Category: Hair Style""}]}]",2,2,"""Cum on Pussy_AI, Pussy Very Cl…","""Cum on Pussy_AI, Pussy Very Cl…"
"""14291""","""Morning Romance""","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""5093"",""American"",[{""7752"",""Category: Misc""}]}, … {""7522"",""Wet Look"",[{""7751"",""Category: Themes""}]}]",2,2,"""Pussy Very Closeup_AI, Wet (Wa…","""Pussy Very Closeup_AI, Wet (Wa…"
"""14654""","""Weekend - Episode 4 - Time For…","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""5169"",""Ass Grabbing"",[{""7753"",""Category: Acts""}]}, … {""7511"",""Wavy Hair"",[{""7770"",""Category: Hair Style""}]}]",2,2,"""Cum on Pussy_AI, Pussy Very Cl…","""Cum on Pussy_AI, Pussy Very Cl…"
"""19444""","""Boss and Employee Celebrate wi…","[{""5054"",""4K Available"",[{""7752"",""Category: Misc""}]}, {""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, … {""6146"",""Woman's Heels"",[{""7762"",""Category: Clothing""}]}]",2,2,"""Cum_AI, Pussy Very Closeup_AI""","""Cum_AI, Pussy Very Closeup_AI"""
"""22436""","""Sun Salutation""","[{""5054"",""4K Available"",[{""7752"",""Category: Misc""}]}, {""5059"",""60 FPS"",[{""7749"",""Category: Shot Type""}]}, … {""7549"",""Workout"",[{""7751"",""Category: Themes""}]}]",2,2,"""Pussy Rubbing_AI, Vaginal Pene…","""Pussy Rubbing_AI, Vaginal Pene…"
…,…,…,…,…,…,…
"""22590""","""Lubricous""","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""8621"",""Ass_AI"",[{""8552"",""AI""}]}, … {""8765"",""Wet Pussy_AI"",[{""8552"",""AI""}]}]",2,2,"""Oiled_AI, Pussy Very Closeup_A…","""Oiled_AI, Pussy Very Closeup_A…"
"""18549""","""Red Hot""","[{""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, {""8621"",""Ass_AI"",[{""8552"",""AI""}]}, … {""8614"",""Thighs_AI"",[{""8552"",""AI""}]}]",2,2,"""Masturbation_AI, Pussy Rubbing…","""Masturbation_AI, Pussy Rubbing…"
"""14656""","""Above""","[{""5060"",""69"",[{""7753"",""Category: Acts""}]}, {""8559"",""AI_Tagged"",[{""8898"",""AI Scene Metadata""}]}, … {""7511"",""Wavy Hair"",[{""7770"",""Category: Hair Style""}]}]",2,2,"""Oiled_AI, Pussy Very Closeup_A…","""Oiled_AI, Pussy Very Closeup_A…"


In [72]:
for selected_scene in scenes_to_be_reset.iter_rows(named=True):
    scene_id = selected_scene["id"]
    scene = stash.find_scene(scene_id, fragment="id title tags { id name parents { id name } } scene_markers { id primary_tag { id name parents { id name } } tags { id name } }")

    # Get all tags from the scene
    all_scene_tags = scene['tags']
    ai_tags = [tag for tag in all_scene_tags if any(parent['id'] == stash_tags.ai["id"] for parent in tag['parents'])]
    ai_tag_ids = [tag['id'] for tag in ai_tags]
    stash_client.bulk_scene_update([scene_id], ai_tag_ids + [stash_tags.ai_tagged["id"]], "REMOVE")

    scene_markers = scene['scene_markers']
    for marker in scene_markers:
        # IF any of the parents of the primary tag is AI, then remove the marker
        if any(parent['id'] == stash_tags.ai["id"] for parent in marker['primary_tag']['parents']):
            stash.destroy_scene_marker(marker["id"])